In [163]:
import pandas as pd
import numpy as np
import regex as re
from io import StringIO
import string

In [164]:
pd_csv = pd.read_csv("output-19-pandas.csv")


In [165]:
pd_csv

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,33,34,35,36,37,38,39,40,41,42
0,0,(Dollars in thousands),NaN,Obligor Name/Interest Rate Range,NaN,Security Type (all restricted unless other...,NaN,Acquisition Date,NaN,Maturity Date,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Medallion Loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,200440.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,Real Cab Corp ##,NaN,Term Loan,NaN,07/20/07,NaN,07/20/17,...,NaN,NaN,2545.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,Real Cab Corp,NaN,Term Loan,NaN,08/19/14,NaN,07/20/17,...,NaN,NaN,481.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,215,"Equity investments, net",NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
216,216,Investment securities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
217,217,"Investment securities, net",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
218,218,"Net Investments ($237,025 pledged as collatera...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [166]:
pd_csv = pd_csv.iloc[:, 1:]
# Resetting index
pd_csv.reset_index()
# Find out where portfolio company is
for col in pd_csv.columns:
    portfolio_index_list = pd_csv.index[pd_csv[col].astype(str).str.contains("Obligor")].to_list()
    if (len(portfolio_index_list) > 0):
        break
portfolio_index = portfolio_index_list[0]
# Checking if all the column are in the row with portfolio and adding them if they are not.
if (pd_csv.iloc[portfolio_index].str.contains('Principal').any() == False):
    for j in range(pd_csv.shape[1]):
        if (pd_csv.iat[portfolio_index - 1, j] == 'Principal'):
            pd_csv.iat[portfolio_index, j] = 'Principal'
        elif (pd_csv.iat[portfolio_index - 1, j] == 'Cost'):
            pd_csv.iat[portfolio_index, j] = 'Cost'
        elif (pd_csv.iat[portfolio_index - 1, j] == 'Fair'):
            pd_csv.iat[portfolio_index, j] = 'Fair Value'
# Removing rows before portfolio index
if (pd_csv.iloc[portfolio_index].str.contains('Company').any() == True):
    pd_csv = pd_csv.iloc[portfolio_index:]



In [167]:
pd_csv.iloc[:,:28]

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,(Dollars in thousands),NaN,Obligor Name/Interest Rate Range,NaN,Security Type (all restricted unless other...,NaN,Acquisition Date,NaN,Maturity Date,NaN,...,NaN,Original Cost of 2016 Acquisitions (5),NaN,NaN,Principal Outstanding,NaN,NaN,Cost (4),NaN,NaN
1,Medallion Loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.76,%,NaN,NaN,12977.0,NaN,NaN,NaN,206576.0
3,NaN,NaN,Real Cab Corp ##,NaN,Term Loan,NaN,07/20/07,NaN,07/20/17,NaN,...,NaN,2.81,%,NaN,NaN,NaN,NaN,NaN,NaN,2545.0
4,NaN,NaN,Real Cab Corp,NaN,Term Loan,NaN,08/19/14,NaN,07/20/17,NaN,...,NaN,3.31,%,NaN,NaN,NaN,NaN,NaN,NaN,481.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,"Equity investments, net",NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,NaN,...,NaN,NaN,1400.0,NaN,NaN,NaN,0,NaN,NaN,NaN
216,Investment securities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
217,"Investment securities, net",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
218,"Net Investments ($237,025 pledged as collatera...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [168]:
pd_csv.isnull().sum()

0     104
1     220
2      90
3     220
4      58
5     219
6      68
7     216
8     120
9     151
10    218
11    129
12    206
13    138
14    219
15    129
16    151
17    143
18    220
19    128
20    119
21    153
22    201
23    203
24    204
25    152
26    154
27    133
28    205
29    201
30    220
31    129
32    208
33    196
34    220
35    129
36    218
37    154
38    220
39    216
40    220
41    154
42    220
dtype: int64

In [169]:
# #Removing extra headers
# second_header = list()
# #Identifying where the second header is
# for index, row in pd_csv.iterrows():
#     if index==1:
#         continue
#     if 'Portfolio Company' in row[0]:
#         second_header.append(index)
# #Dropping the second header
# try:
#     print("Second Header",second_header)
#     pd_csv.drop(second_header, inplace=True)
# except:
#     print("No second header")

In [170]:
pd_csv.shape[1]

43

In [171]:
# Identifying levels in the dataframe
listOfLevels = list()
for i in range(0,pd_csv.shape[0]):
    for j in range(2):
        if (pd.notnull(pd_csv.iat[i, j])):
            # print(pd_csv.iat[i, j])
            if ((('investments' in str(pd_csv.iat[i, j])) or ('Investment' in str(pd_csv.iat[i, j]))
                or ('Loan' in str(pd_csv.iat[i, j]))) and
                    ('Subtotal' not in pd_csv.iat[i, j]) and
                    ('Total' not in pd_csv.iat[i, j])):
                listOfLevels.append(i)

In [172]:
listOfLevels

[1,
 40,
 142,
 146,
 148,
 149,
 173,
 177,
 179,
 180,
 187,
 197,
 198,
 199,
 200,
 205,
 215,
 216,
 217,
 218]

In [173]:
pd_csv

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,(Dollars in thousands),NaN,Obligor Name/Interest Rate Range,NaN,Security Type (all restricted unless other...,NaN,Acquisition Date,NaN,Maturity Date,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Medallion Loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,200440.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,Real Cab Corp ##,NaN,Term Loan,NaN,07/20/07,NaN,07/20/17,NaN,...,NaN,NaN,2545.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,Real Cab Corp,NaN,Term Loan,NaN,08/19/14,NaN,07/20/17,NaN,...,NaN,NaN,481.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,"Equity investments, net",NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
216,Investment securities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
217,"Investment securities, net",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
218,"Net Investments ($237,025 pledged as collatera...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [174]:
# Adding the Levels to an array
newValues = []
newValues.append("Levels")
for i in range(len(listOfLevels)):
    beginIndex = listOfLevels[i]
    if (i == len(listOfLevels) - 1):
        endIndex = len(pd_csv)
    else:
        endIndex = listOfLevels[i + 1]
    value = pd_csv.iat[beginIndex, 0]
    for x in range(beginIndex, endIndex):
        newValues.append(value)
# Adding the array to the df
pd_csv.insert(0, 'Levels', newValues)

In [175]:
newValues

['Levels',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Medallion Loans',
 'Commercial Loans',
 'Commercial Loans',
 'Commercial Loans',
 'Commercial Loans',
 'Commercial Loans',
 'Commercial Loans',
 'Commercial Loans',
 'Commercial Loans',
 'Commercial Loans',
 'Commercial Loans',

In [176]:
pd_csv

,Levels,0,1,2,3,4,5,6,7,8,...,33,34,35,36,37,38,39,40,41,42
0,Levels,(Dollars in thousands),NaN,Obligor Name/Interest Rate Range,NaN,Security Type (all restricted unless other...,NaN,Acquisition Date,NaN,Maturity Date,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Medallion Loans,Medallion Loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Medallion Loans,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,200440.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Medallion Loans,NaN,NaN,Real Cab Corp ##,NaN,Term Loan,NaN,07/20/07,NaN,07/20/17,...,NaN,NaN,2545.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Medallion Loans,NaN,NaN,Real Cab Corp,NaN,Term Loan,NaN,08/19/14,NaN,07/20/17,...,NaN,NaN,481.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,"Equity investments, net","Equity investments, net",NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
216,Investment securities,Investment securities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
217,"Investment securities, net","Investment securities, net",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
218,"Net Investments ($237,025 pledged as collatera...","Net Investments ($237,025 pledged as collatera...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [177]:
# Finding Subtotals
pd_csv.reset_index()
subtotal_list = list()
total_list = list()
for i in range(pd_csv.shape[0]):
    for j in range(pd_csv.shape[1]):
        if (('Subtotal' in str(pd_csv.iat[i, j]))):
            # print(pd_csv.iloc[i,:])
            subtotal_list.append(i)

print("Subtotal", subtotal_list)
# Dropping the subtotal rows
pd_csv.drop(pd_csv.index[subtotal_list], inplace=True)

# Finding Totals
total_list = list()
for i in range(pd_csv.shape[0]):
    for j in range(pd_csv.shape[1]):
        if ('Total' in str(pd_csv.iat[i, j])):
            total_list.append(i)
            
# Dropping the total rows
pd_csv.drop(pd_csv.index[total_list], inplace=True)

# Dropping everything after the total rows
# try:
#     pd_csv = pd_csv.iloc[:total_list[-1]]
# except:
#     print("Total not found")
    

for j in range(pd_csv.shape[1]):
    print(pd_csv.iat[0,j])
    if ('TypeofInvestment' in str(pd_csv.iat[0,j]).translate({ord(c): None for c in string.whitespace})):
        type_of_investment_index = j
    elif ('Obligor' in str(pd_csv.iat[0,j]).translate({ord(c): None for c in string.whitespace})):
        print(str(pd_csv.iat[0,j]).translate({ord(c): None for c in string.whitespace}))
        type_of_investment_index = j
        break

Subtotal []
Levels
(Dollars in  thousands)
nan
Obligor  Name/Interest Rate  Range
ObligorName/InterestRateRange


In [178]:
type_of_investment_index

3

In [179]:
pd_csv

,Levels,0,1,2,3,4,5,6,7,8,...,33,34,35,36,37,38,39,40,41,42
0,Levels,(Dollars in thousands),NaN,Obligor Name/Interest Rate Range,NaN,Security Type (all restricted unless other...,NaN,Acquisition Date,NaN,Maturity Date,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Medallion Loans,Medallion Loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Medallion Loans,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,200440.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Medallion Loans,NaN,NaN,Real Cab Corp ##,NaN,Term Loan,NaN,07/20/07,NaN,07/20/17,...,NaN,NaN,2545.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Medallion Loans,NaN,NaN,Real Cab Corp,NaN,Term Loan,NaN,08/19/14,NaN,07/20/17,...,NaN,NaN,481.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,"Equity investments, net","Equity investments, net",NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
216,Investment securities,Investment securities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
217,"Investment securities, net","Investment securities, net",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
218,"Net Investments ($237,025 pledged as collatera...","Net Investments ($237,025 pledged as collatera...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [180]:
# print(type_of_investment_index)
# Adding company for the middle values where it might be missed out
pd_csv = pd_csv.replace("+",np.nan)


for i in range(2, pd_csv.shape[0]):
    for j in range(type_of_investment_index):
        row_alphabet_check = False
        if (pd.isnull(pd_csv.iat[i, j]) or (pd_csv.iat[i, j] == 'nan')):
            # print("Index",i,j)
            if (i > 0):
                # print(pd_csv.iat[i-1,j])
                if (pd.isnull(pd_csv.iat[i - 1, j])):
                    1
                else:
                    if (re.search('[a-zA-Z]', str(pd_csv.iat[i - 1, j]))):
                        for k in range(pd_csv.shape[1]):
                            #print(pd_csv.iat[i, 0])
                            if (re.search('[a-zA-Z]', str(pd_csv.iat[i, k])) and pd.notnull(pd_csv.iat[i, k]) ):
                               
                                #print(pd_csv.iloc[i])
                                # print("Alpha Check",pd_csv.iat[i,k])
                                if(j>1):
                                    if(pd_csv.iat[i, 1]==pd_csv.iat[i-1, 1]):
                                        row_alphabet_check = True
                                else:
                                    row_alphabet_check = True   
                                break
                        if (row_alphabet_check):
                            # print(pd_csv.iat[i-1,j])
                            pd_csv.iat[i, j] = pd_csv.iat[i - 1, j]



In [181]:
pd_csv[:60]

,Levels,0,1,2,3,4,5,6,7,8,...,33,34,35,36,37,38,39,40,41,42
0,Levels,(Dollars in thousands),NaN,Obligor Name/Interest Rate Range,NaN,Security Type (all restricted unless other...,NaN,Acquisition Date,NaN,Maturity Date,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Medallion Loans,Medallion Loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Medallion Loans,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,200440.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Medallion Loans,New York,NaN,Real Cab Corp ##,NaN,Term Loan,NaN,07/20/07,NaN,07/20/17,...,NaN,NaN,2545.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Medallion Loans,New York,NaN,Real Cab Corp,NaN,Term Loan,NaN,08/19/14,NaN,07/20/17,...,NaN,NaN,481.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Medallion Loans,New York,NaN,Real Cab Corp ##,NaN,Term Loan,NaN,07/20/07,NaN,07/20/17,...,NaN,NaN,350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Medallion Loans,New York,NaN,Sean Cab Corp ##,NaN,Term Loan,NaN,12/09/11,NaN,11/23/18,...,NaN,NaN,3297.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Medallion Loans,New York,NaN,Slo Cab Corp ##,NaN,Term Loan,NaN,07/20/07,NaN,07/20/17,...,NaN,NaN,1527.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Medallion Loans,New York,NaN,Slo Cab Corp ##,NaN,Term Loan,NaN,08/19/14,NaN,07/20/17,...,NaN,NaN,289.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Medallion Loans,New York,NaN,Slo Cab Corp ##,NaN,Term Loan,NaN,07/20/07,NaN,07/20/17,...,NaN,NaN,210.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [182]:
# Identifying index of Interest
i = 0
for j in range(pd_csv.shape[1]):
    if ( ('Interest' in str(pd_csv.iat[i, j])) or ('Rate' in str(pd_csv.iat[i, j])) 
       or  ('Security' in str(pd_csv.iat[i, j]))) :
        principal_index = j
        break
# Extracting the column data after principal
pd_csv_principal = pd_csv
pd_csv_principal = pd_csv_principal.replace("%",np.nan)
# Sweeping everyhing to left in extracted df
pd_csv_principal = pd.read_csv(StringIO(re.sub(',+', ',', pd_csv_principal.to_csv())))
pd_csv_principal = pd_csv_principal.iloc[:, 1:]
# Dropping columns with Nulls
perc = 95  # Like N %.
min_count = int(((100 - perc) / 100) * pd_csv_principal.shape[0] + 1)
pd_csv_principal = pd_csv_principal.dropna(axis=1,
                                           thresh=min_count)

pd_csv_principal.replace('na', np.nan, inplace=True)




In [183]:
pd_csv_principal

,Levels,0,1,2,3,4,5,6,7,8,9,10,11
0,Levels,(Dollars in thousands),Obligor Name/Interest Rate Range,Security Type (all restricted unless other...,Acquisition Date,Maturity Date,No. of Invest.,% of Net Assets,Interest Rate (1),Original Cost of 2016 Acquisitions (5),Principal Outstanding,Cost (4),Fair Value
1,Medallion Loans,Medallion Loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Medallion Loans,New York,386.0,71,3.76,12977.0,206576.0,206404.0,200440.0,NaN,NaN,NaN,NaN
3,Medallion Loans,New York,Real Cab Corp ##,Term Loan,07/20/07,07/20/17,1.0,1,2.81,2545.0,2545.0,2545.0,NaN
4,Medallion Loans,New York,Real Cab Corp,Term Loan,08/19/14,07/20/17,1.0,*,3.31,481.0,481.0,481.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,"Equity investments, net","Equity investments, net",28.0,3.0,0.66,1400.0,0,4760,8163.0,NaN,NaN,NaN,NaN
209,Investment securities,Investment securities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210,"Investment securities, net","Investment securities, net",0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
211,"Net Investments ($237,025 pledged as collatera...","Net Investments ($237,025 pledged as collatera...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [184]:
pd_csv

,Levels,0,1,2,3,4,5,6,7,8,...,33,34,35,36,37,38,39,40,41,42
0,Levels,(Dollars in thousands),NaN,Obligor Name/Interest Rate Range,NaN,Security Type (all restricted unless other...,NaN,Acquisition Date,NaN,Maturity Date,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Medallion Loans,Medallion Loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Medallion Loans,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,200440.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Medallion Loans,New York,NaN,Real Cab Corp ##,NaN,Term Loan,NaN,07/20/07,NaN,07/20/17,...,NaN,NaN,2545.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Medallion Loans,New York,NaN,Real Cab Corp,NaN,Term Loan,NaN,08/19/14,NaN,07/20/17,...,NaN,NaN,481.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,"Equity investments, net","Equity investments, net",NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
216,Investment securities,Investment securities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
217,"Investment securities, net","Investment securities, net",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
218,"Net Investments ($237,025 pledged as collatera...","Net Investments ($237,025 pledged as collatera...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [185]:
# Concatting everything together
pd_csv = pd_csv_principal

In [186]:
pd_csv[:60]

,Levels,0,1,2,3,4,5,6,7,8,9,10,11
0,Levels,(Dollars in thousands),Obligor Name/Interest Rate Range,Security Type (all restricted unless other...,Acquisition Date,Maturity Date,No. of Invest.,% of Net Assets,Interest Rate (1),Original Cost of 2016 Acquisitions (5),Principal Outstanding,Cost (4),Fair Value
1,Medallion Loans,Medallion Loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Medallion Loans,New York,386.0,71,3.76,12977.0,206576.0,206404.0,200440.0,NaN,NaN,NaN,NaN
3,Medallion Loans,New York,Real Cab Corp ##,Term Loan,07/20/07,07/20/17,1.0,1,2.81,2545.0,2545.0,2545.0,NaN
4,Medallion Loans,New York,Real Cab Corp,Term Loan,08/19/14,07/20/17,1.0,*,3.31,481.0,481.0,481.0,NaN
5,Medallion Loans,New York,Real Cab Corp ##,Term Loan,07/20/07,07/20/17,1.0,*,2.81,350.0,350.0,350.0,NaN
6,Medallion Loans,New York,Sean Cab Corp ##,Term Loan,12/09/11,11/23/18,1.0,1,4.63,3297.0,3297.0,3297.0,NaN
7,Medallion Loans,New York,Slo Cab Corp ##,Term Loan,07/20/07,07/20/17,1.0,1,2.81,1527.0,1527.0,1527.0,NaN
8,Medallion Loans,New York,Slo Cab Corp ##,Term Loan,08/19/14,07/20/17,1.0,*,3.31,289.0,289.0,289.0,NaN
9,Medallion Loans,New York,Slo Cab Corp ##,Term Loan,07/20/07,07/20/17,1.0,*,2.81,210.0,210.0,210.0,NaN


In [187]:
pd_csv

,Levels,0,1,2,3,4,5,6,7,8,9,10,11
0,Levels,(Dollars in thousands),Obligor Name/Interest Rate Range,Security Type (all restricted unless other...,Acquisition Date,Maturity Date,No. of Invest.,% of Net Assets,Interest Rate (1),Original Cost of 2016 Acquisitions (5),Principal Outstanding,Cost (4),Fair Value
1,Medallion Loans,Medallion Loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Medallion Loans,New York,386.0,71,3.76,12977.0,206576.0,206404.0,200440.0,NaN,NaN,NaN,NaN
3,Medallion Loans,New York,Real Cab Corp ##,Term Loan,07/20/07,07/20/17,1.0,1,2.81,2545.0,2545.0,2545.0,NaN
4,Medallion Loans,New York,Real Cab Corp,Term Loan,08/19/14,07/20/17,1.0,*,3.31,481.0,481.0,481.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,"Equity investments, net","Equity investments, net",28.0,3.0,0.66,1400.0,0,4760,8163.0,NaN,NaN,NaN,NaN
209,Investment securities,Investment securities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210,"Investment securities, net","Investment securities, net",0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
211,"Net Investments ($237,025 pledged as collatera...","Net Investments ($237,025 pledged as collatera...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [188]:
# rows_with_nums = list()
# rows_with_nums.append(0)
# rows_with_nums.append(1)
# for i in range(0,pd_csv.shape[0]):
#     if(pd_csv.iloc[i,-6:].str.isdigit().any()):
#         rows_with_nums.append(i)
# len(rows_with_nums)

In [189]:
#pd_csv = pd_csv.iloc[rows_with_nums]

In [190]:
pd_csv

,Levels,0,1,2,3,4,5,6,7,8,9,10,11
0,Levels,(Dollars in thousands),Obligor Name/Interest Rate Range,Security Type (all restricted unless other...,Acquisition Date,Maturity Date,No. of Invest.,% of Net Assets,Interest Rate (1),Original Cost of 2016 Acquisitions (5),Principal Outstanding,Cost (4),Fair Value
1,Medallion Loans,Medallion Loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Medallion Loans,New York,386.0,71,3.76,12977.0,206576.0,206404.0,200440.0,NaN,NaN,NaN,NaN
3,Medallion Loans,New York,Real Cab Corp ##,Term Loan,07/20/07,07/20/17,1.0,1,2.81,2545.0,2545.0,2545.0,NaN
4,Medallion Loans,New York,Real Cab Corp,Term Loan,08/19/14,07/20/17,1.0,*,3.31,481.0,481.0,481.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,"Equity investments, net","Equity investments, net",28.0,3.0,0.66,1400.0,0,4760,8163.0,NaN,NaN,NaN,NaN
209,Investment securities,Investment securities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210,"Investment securities, net","Investment securities, net",0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
211,"Net Investments ($237,025 pledged as collatera...","Net Investments ($237,025 pledged as collatera...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [191]:
# Sweeping to left for member units
# pd_csv_sweep_left = pd_csv.iloc[:, :(principal_index + 2)]
# pd_csv_sweep_left = pd.read_csv(StringIO(re.sub(',+', ',', pd_csv_sweep_left.to_csv())))
# pd_csv_sweep_right = pd_csv.iloc[:, principal_index + 2:]
# pd_csv = pd.concat([pd_csv_sweep_left, pd_csv_sweep_right], axis=1)

perc = 100  # Like N %
min_count = int(((100 - perc) / 100) * pd_csv.shape[0] + 1)
# Dropping columns which have 100% NaN
pd_csv = pd_csv.dropna(axis=1,
                       thresh=min_count)

new_header = pd_csv.iloc[0]  # grab the first row for the header
pd_csv = pd_csv[1:]  # take the data less the header row
pd_csv.columns = new_header.str.replace(r"\(\d+\)", "")  # set the header row as the df header

# Dropping columns which NaN as column names
pd_csv = pd_csv.loc[:, pd_csv.columns.notna()]
pd_csv = pd_csv.dropna(thresh=len(pd_csv.columns) - 4)


C:\Users\jaiso\AppData\Local\Temp\ipykernel_19612\1961526554.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  pd_csv.columns = new_header.str.replace(r"\(\d+\)", "")  # set the header row as the df header


In [192]:
company_cols = [col for col in pd_csv.columns if 'Portfolio' in col]
company_cols

[]

In [193]:
try:
    level_rows_to_drop = pd_csv.index[pd_csv[company_cols[0]]==pd_csv['Levels']].to_list()
    pd_csv.drop(level_rows_to_drop, inplace=True) 
    print(level_rows_to_drop)
except:
    print("No Levels to drop")

No Levels to drop


In [194]:
pd_csv

,Levels,(Dollars in thousands),Obligor Name/Interest Rate Range,Security Type (all restricted unless otherwise noted),Acquisition Date,Maturity Date,No. of Invest.,% of Net Assets,Interest Rate,Original Cost of 2016 Acquisitions,Principal Outstanding,Cost,Fair Value
2,Medallion Loans,New York,386.0,71,3.76,12977.0,206576.0,206404.0,200440.0,NaN,NaN,NaN,NaN
3,Medallion Loans,New York,Real Cab Corp ##,Term Loan,07/20/07,07/20/17,1.0,1,2.81,2545.0,2545.0,2545.0,NaN
4,Medallion Loans,New York,Real Cab Corp,Term Loan,08/19/14,07/20/17,1.0,*,3.31,481.0,481.0,481.0,NaN
5,Medallion Loans,New York,Real Cab Corp ##,Term Loan,07/20/07,07/20/17,1.0,*,2.81,350.0,350.0,350.0,NaN
6,Medallion Loans,New York,Sean Cab Corp ##,Term Loan,12/09/11,11/23/18,1.0,1,4.63,3297.0,3297.0,3297.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,Investment Castings,Surgical Supplies Manufacturer,"Respiratory Technologies, Inc.","Warrant for 38,000 Common shares",04/25/12,5th anniversary of note paid in full,1,*,0.0,0.0,0.0,NaN,NaN
206,Investment Castings,Hobbyists’ Supplies Merch. Wholesale,"Classic Brand, LLC","Warrant for 300,000 Class A units",01/08/16,01/08/26,1,*,0.0,0.0,0.0,0.0,NaN
207,Investment Castings,Various Other #,**,* Various,09/10/98 to 7/24/15,None to 2/5/23,12,*,2.29,250.0,1366.0,1309.0,NaN
208,"Equity investments, net","Equity investments, net",28.0,3.0,0.66,1400.0,0,4760,8163.0,NaN,NaN,NaN,NaN


In [195]:
investment_cols = [col for col in pd_csv.columns if 'Investment' in col]
investment_cols

[]

In [196]:
principal_cols = [col for col in pd_csv.columns if 'Principal' in col]
principal_cols

['Principal  Outstanding']

In [197]:
try:
    investment_empty = pd_csv.index[(pd_csv['Principal'].str.contains('[a-zA-Z]')) & (pd_csv[investment_cols[0]].isnull())].to_list()
    print(investment_empty)
    pd_csv.loc[investment_empty, [investment_cols[0],principal_cols[0]]] = pd_csv.loc[investment_empty, [principal_cols[0],investment_cols[0]]].values
except:
    print("Investment Empty")

Investment Empty


In [198]:
#Rate column has investment values
rate_cols = [col for col in pd_csv.columns if 'Rate' in col]
try:
    rate_no_num = pd_csv.index[(pd_csv[rate_cols[0]].str.contains('[a-zA-Z]')) & (pd_csv[investment_cols[0]].isnull())].to_list()
    pd_csv.loc[rate_no_num, [investment_cols[0],rate_cols[0]]] = pd_csv.loc[rate_no_num, [rate_cols[0],investment_cols[0]]].values
except:
    print("No rate")

No rate


In [199]:
pd_csv

,Levels,(Dollars in thousands),Obligor Name/Interest Rate Range,Security Type (all restricted unless otherwise noted),Acquisition Date,Maturity Date,No. of Invest.,% of Net Assets,Interest Rate,Original Cost of 2016 Acquisitions,Principal Outstanding,Cost,Fair Value
2,Medallion Loans,New York,386.0,71,3.76,12977.0,206576.0,206404.0,200440.0,NaN,NaN,NaN,NaN
3,Medallion Loans,New York,Real Cab Corp ##,Term Loan,07/20/07,07/20/17,1.0,1,2.81,2545.0,2545.0,2545.0,NaN
4,Medallion Loans,New York,Real Cab Corp,Term Loan,08/19/14,07/20/17,1.0,*,3.31,481.0,481.0,481.0,NaN
5,Medallion Loans,New York,Real Cab Corp ##,Term Loan,07/20/07,07/20/17,1.0,*,2.81,350.0,350.0,350.0,NaN
6,Medallion Loans,New York,Sean Cab Corp ##,Term Loan,12/09/11,11/23/18,1.0,1,4.63,3297.0,3297.0,3297.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,Investment Castings,Surgical Supplies Manufacturer,"Respiratory Technologies, Inc.","Warrant for 38,000 Common shares",04/25/12,5th anniversary of note paid in full,1,*,0.0,0.0,0.0,NaN,NaN
206,Investment Castings,Hobbyists’ Supplies Merch. Wholesale,"Classic Brand, LLC","Warrant for 300,000 Class A units",01/08/16,01/08/26,1,*,0.0,0.0,0.0,0.0,NaN
207,Investment Castings,Various Other #,**,* Various,09/10/98 to 7/24/15,None to 2/5/23,12,*,2.29,250.0,1366.0,1309.0,NaN
208,"Equity investments, net","Equity investments, net",28.0,3.0,0.66,1400.0,0,4760,8163.0,NaN,NaN,NaN,NaN


In [200]:
#Maturity switching when Principal data in maturity column
try:
    maturity_cols = [col for col in pd_csv.columns if 'Maturity' in col]
    maturity_index = pd_csv.columns.get_loc(maturity_cols[0])
    maturity_nan_index = pd_csv.index[~(pd_csv['Maturity'].astype(str).str.contains("/") | pd_csv['Maturity'].astype(str).str.contains("-"))].to_list()
    pd_csv.loc[maturity_nan_index,'Maturity':] = pd_csv.loc[maturity_nan_index,'Maturity':].shift(1,axis=1)
except:
    print("No Principal")

No Principal


In [201]:
pd_csv

,Levels,(Dollars in thousands),Obligor Name/Interest Rate Range,Security Type (all restricted unless otherwise noted),Acquisition Date,Maturity Date,No. of Invest.,% of Net Assets,Interest Rate,Original Cost of 2016 Acquisitions,Principal Outstanding,Cost,Fair Value
2,Medallion Loans,New York,386.0,71,3.76,12977.0,206576.0,206404.0,200440.0,NaN,NaN,NaN,NaN
3,Medallion Loans,New York,Real Cab Corp ##,Term Loan,07/20/07,07/20/17,1.0,1,2.81,2545.0,2545.0,2545.0,NaN
4,Medallion Loans,New York,Real Cab Corp,Term Loan,08/19/14,07/20/17,1.0,*,3.31,481.0,481.0,481.0,NaN
5,Medallion Loans,New York,Real Cab Corp ##,Term Loan,07/20/07,07/20/17,1.0,*,2.81,350.0,350.0,350.0,NaN
6,Medallion Loans,New York,Sean Cab Corp ##,Term Loan,12/09/11,11/23/18,1.0,1,4.63,3297.0,3297.0,3297.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,Investment Castings,Surgical Supplies Manufacturer,"Respiratory Technologies, Inc.","Warrant for 38,000 Common shares",04/25/12,5th anniversary of note paid in full,1,*,0.0,0.0,0.0,NaN,NaN
206,Investment Castings,Hobbyists’ Supplies Merch. Wholesale,"Classic Brand, LLC","Warrant for 300,000 Class A units",01/08/16,01/08/26,1,*,0.0,0.0,0.0,0.0,NaN
207,Investment Castings,Various Other #,**,* Various,09/10/98 to 7/24/15,None to 2/5/23,12,*,2.29,250.0,1366.0,1309.0,NaN
208,"Equity investments, net","Equity investments, net",28.0,3.0,0.66,1400.0,0,4760,8163.0,NaN,NaN,NaN,NaN


In [202]:
pd_csv[(pd_csv.iloc[: , -1].isnull()) & (pd_csv.iloc[: , -2].notnull())]

,Levels,(Dollars in thousands),Obligor Name/Interest Rate Range,Security Type (all restricted unless otherwise noted),Acquisition Date,Maturity Date,No. of Invest.,% of Net Assets,Interest Rate,Original Cost of 2016 Acquisitions,Principal Outstanding,Cost,Fair Value
3,Medallion Loans,New York,Real Cab Corp ##,Term Loan,07/20/07,07/20/17,1.0,1,2.81,2545.0,2545.0,2545.0,NaN
4,Medallion Loans,New York,Real Cab Corp,Term Loan,08/19/14,07/20/17,1.0,*,3.31,481.0,481.0,481.0,NaN
5,Medallion Loans,New York,Real Cab Corp ##,Term Loan,07/20/07,07/20/17,1.0,*,2.81,350.0,350.0,350.0,NaN
6,Medallion Loans,New York,Sean Cab Corp ##,Term Loan,12/09/11,11/23/18,1.0,1,4.63,3297.0,3297.0,3297.0,NaN
7,Medallion Loans,New York,Slo Cab Corp ##,Term Loan,07/20/07,07/20/17,1.0,1,2.81,1527.0,1527.0,1527.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,Investment Castings,Hand Tool Manufacturer,"Stride Tool Holdings, LLC",7.14% of Series A-2 LLC units,04/05/16,None,1,*,0.0,500.0,500.0,500.0,NaN
203,Investment Castings,Envirnomental Consulting Services,"Northern Technologies, LLC",7.7% of LLC units,01/29/16,None,1,*,0.0,300.0,300.0,300.0,NaN
204,Investment Castings,Paper Tapes Manufacturer,"Liberty Paper Products Acquisition, LLC",100% of Series A preferred LLC units - 12% total,06/09/16,None,1,*,0.0,350.0,350.0,350.0,NaN
206,Investment Castings,Hobbyists’ Supplies Merch. Wholesale,"Classic Brand, LLC","Warrant for 300,000 Class A units",01/08/16,01/08/26,1,*,0.0,0.0,0.0,0.0,NaN


In [203]:
#Indexes where Fair Value is Empty
fair_value_nan = pd_csv.index[(pd_csv.iloc[: , -1].isnull()) & (pd_csv.iloc[: , -2].notnull())].to_list()
fair_value_nan

[3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 33,
 37,
 38,
 43,
 45,
 49,
 51,
 53,
 55,
 57,
 59,
 61,
 62,
 64,
 65,
 66,
 67,
 68,
 72,
 73,
 75,
 76,
 78,
 79,
 81,
 83,
 87,
 89,
 91,
 93,
 95,
 97,
 99,
 100,
 102,
 103,
 104,
 105,
 106,
 110,
 111,
 113,
 114,
 116,
 117,
 119,
 121,
 122,
 124,
 125,
 126,
 128,
 134,
 135,
 136,
 137,
 146,
 149,
 150,
 152,
 153,
 154,
 156,
 162,
 163,
 164,
 165,
 174,
 184,
 185,
 186,
 188,
 189,
 202,
 203,
 204,
 206,
 207]

In [204]:
original_cost_cols = [col for col in pd_csv.columns if 'Original  Cost of' in col]
original_cost_cols[0]

'Original  Cost of 2016  Acquisitions  '

In [205]:
try:
    #Shifting the columns when fair value is empty
    pd_csv.loc[fair_value_nan,original_cost_cols[0]:] = pd_csv.loc[fair_value_nan,original_cost_cols[0]:].shift(1,axis=1)
except:
    print("Fair Value not present, Principal Column shifting")

In [206]:
#Indexes where Maturity and Principal Value is Empty
try:
    maturity_principal_nan = pd_csv.index[(pd_csv[maturity_cols[0]].isnull()) & 
                                                       (pd_csv[principal_cols[0]].isnull()) &
                                                       (pd_csv[rate_cols[0]].notnull())].to_list()
    pd_csv.loc[maturity_principal_nan, [rate_cols[0],principal_cols[0]]] = \
    pd_csv.loc[maturity_principal_nan, [principal_cols[0],rate_cols[0]]].values

except:
    print("No Rate")


In [207]:
try:
    company_cols = [col for col in pd_csv.columns if 'Company' in col]
    company_cols[0]
except:
    print("Company")

Company


In [208]:
#Finding where Money Market is there on the last row
try:
    money_market_last_index = pd_csv.index[pd_csv[company_cols[0]].str.contains('Money Market')].to_list()[-1]
    pd_csv = pd_csv.loc[:money_market_last_index]
    investment_cols = [col for col in pd_csv.columns if 'Investment' in col]
    principal_investment = pd_csv.columns.get_loc(investment_cols[0])
    print(principal_investment)
    # Extracting the column data after principal
    pd_csv_investment = pd_csv.iloc[:, principal_investment+1:]
    pd_csv_investment
    
    perc = 80  # Like N %.
    min_count = int(((100 - perc) / 100) * pd_csv_principal.shape[1] + 1)
    num_of_nulls_allowed = pd_csv_investment.shape[1]- min_count
    num_of_nulls_allowed
    
    pd_csv = pd_csv.reset_index(drop=True)
    #Finding out the values where the Type of Investment is overflowing to the next row
    interest_investment_index = list()
    for i in range(1,pd_csv.shape[0]):
        if(pd_csv_investment.iloc[i].isnull().sum()>num_of_nulls_allowed):
            print(pd_csv.iloc[i])
            interest_investment_index.append(i)
except:
    print("Money market not present")


Money market not present


In [209]:
#Printing those values out
#interest_investment_index

In [210]:
# Appending the overflowing values to the row above it
try:
    for i in interest_investment_index:
        pd_csv.iat[i-1, principal_investment] = pd_csv.iat[i-1, principal_investment]+pd_csv.iat[i, principal_investment]
        print(pd_csv.iat[i-1, principal_investment])
        #pd_csv.iat[i, j] = pd_csv.iat[i - 1, j]
except:
    print("No Money Market")
    


No Money Market


In [211]:
#Dropping the rows which had the overflowing Type of Investment valuse
#pd_csv.drop(interest_investment_index, inplace=True, axis=0)


In [212]:
pd_csv.isnull().sum()

0
Levels                                                          0
(Dollars in  thousands)                                         0
Obligor  Name/Interest Rate  Range                              0
Security  Type (all  restricted  unless  otherwise  noted)      0
Acquisition  Date                                               0
Maturity  Date                                                  0
No. of  Invest.                                                 0
% of  Net  Assets                                               0
Interest  Rate                                                  0
Original  Cost of 2016  Acquisitions                          114
Principal  Outstanding                                          8
Cost                                                           38
Fair  Value                                                    38
dtype: int64

In [213]:
pd_csv

,Levels,(Dollars in thousands),Obligor Name/Interest Rate Range,Security Type (all restricted unless otherwise noted),Acquisition Date,Maturity Date,No. of Invest.,% of Net Assets,Interest Rate,Original Cost of 2016 Acquisitions,Principal Outstanding,Cost,Fair Value
2,Medallion Loans,New York,386.0,71,3.76,12977.0,206576.0,206404.0,200440.0,NaN,NaN,NaN,NaN
3,Medallion Loans,New York,Real Cab Corp ##,Term Loan,07/20/07,07/20/17,1.0,1,2.81,NaN,2545.0,2545.0,2545.0
4,Medallion Loans,New York,Real Cab Corp,Term Loan,08/19/14,07/20/17,1.0,*,3.31,NaN,481.0,481.0,481.0
5,Medallion Loans,New York,Real Cab Corp ##,Term Loan,07/20/07,07/20/17,1.0,*,2.81,NaN,350.0,350.0,350.0
6,Medallion Loans,New York,Sean Cab Corp ##,Term Loan,12/09/11,11/23/18,1.0,1,4.63,NaN,3297.0,3297.0,3297.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,Investment Castings,Surgical Supplies Manufacturer,"Respiratory Technologies, Inc.","Warrant for 38,000 Common shares",04/25/12,5th anniversary of note paid in full,1,*,0.0,0.0,0.0,NaN,NaN
206,Investment Castings,Hobbyists’ Supplies Merch. Wholesale,"Classic Brand, LLC","Warrant for 300,000 Class A units",01/08/16,01/08/26,1,*,0.0,NaN,0.0,0.0,0.0
207,Investment Castings,Various Other #,**,* Various,09/10/98 to 7/24/15,None to 2/5/23,12,*,2.29,NaN,250.0,1366.0,1309.0
208,"Equity investments, net","Equity investments, net",28.0,3.0,0.66,1400.0,0,4760,8163.0,NaN,NaN,NaN,NaN


In [214]:
pd_csv[(pd_csv.iloc[: , -1].isnull()) & (pd_csv.iloc[: , -2].isnull()) & (pd_csv.iloc[: , -3].isnull()) ]

,Levels,(Dollars in thousands),Obligor Name/Interest Rate Range,Security Type (all restricted unless otherwise noted),Acquisition Date,Maturity Date,No. of Invest.,% of Net Assets,Interest Rate,Original Cost of 2016 Acquisitions,Principal Outstanding,Cost,Fair Value
2,Medallion Loans,New York,386.0,71,3.76,12977.0,206576.0,206404.0,200440.0,NaN,NaN,NaN,NaN
32,Medallion Loans,Chicago,111.0,13,4.57,109.0,38562.0,38432.0,35344.0,NaN,NaN,NaN,NaN
144,Investment in Medallion Bank and other control...,Investment in Medallion Bank and other control...,5,74.0,3.88,0.0,0,154656,208098,NaN,NaN,NaN,NaN
172,Investment in Medallion Bank and other control...,Investment in Medallion Bank and other control...,5,74.0,3.88,0.0,0,154656,208098,NaN,NaN,NaN,NaN
190,"Equity investments, net","Equity investments, net",28.0,3.0,0.66,1400.0,0,4760,8163.0,NaN,NaN,NaN,NaN
194,"Net Investments ($237,025 pledged as collatera...","Net Investments ($237,025 pledged as collatera...",780.0,209.0,5.36,34440.0,383210,542005,585628.0,NaN,NaN,NaN,NaN
208,"Equity investments, net","Equity investments, net",28.0,3.0,0.66,1400.0,0,4760,8163.0,NaN,NaN,NaN,NaN
212,"Net Investments ($237,025 pledged as collatera...","Net Investments ($237,025 pledged as collatera...",780.0,209.0,5.36,34440.0,383210,542005,585628.0,NaN,NaN,NaN,NaN


In [215]:
obligor_cols = [col for col in pd_csv.columns if 'Obligor' in col]
obligor_cols[0]

'Obligor  Name/Interest Rate  Range'

In [216]:
pd_csv[(pd_csv.iloc[: , -1].isnull()) & (pd_csv.iloc[: , -2].isnull()) & (pd_csv.iloc[: , -3].isnull())  & (pd_csv.iloc[: , -4].isnull()) ]

,Levels,(Dollars in thousands),Obligor Name/Interest Rate Range,Security Type (all restricted unless otherwise noted),Acquisition Date,Maturity Date,No. of Invest.,% of Net Assets,Interest Rate,Original Cost of 2016 Acquisitions,Principal Outstanding,Cost,Fair Value
2,Medallion Loans,New York,386.0,71,3.76,12977.0,206576.0,206404.0,200440.0,NaN,NaN,NaN,NaN
32,Medallion Loans,Chicago,111.0,13,4.57,109.0,38562.0,38432.0,35344.0,NaN,NaN,NaN,NaN
144,Investment in Medallion Bank and other control...,Investment in Medallion Bank and other control...,5,74.0,3.88,0.0,0,154656,208098,NaN,NaN,NaN,NaN
172,Investment in Medallion Bank and other control...,Investment in Medallion Bank and other control...,5,74.0,3.88,0.0,0,154656,208098,NaN,NaN,NaN,NaN
190,"Equity investments, net","Equity investments, net",28.0,3.0,0.66,1400.0,0,4760,8163.0,NaN,NaN,NaN,NaN
194,"Net Investments ($237,025 pledged as collatera...","Net Investments ($237,025 pledged as collatera...",780.0,209.0,5.36,34440.0,383210,542005,585628.0,NaN,NaN,NaN,NaN
208,"Equity investments, net","Equity investments, net",28.0,3.0,0.66,1400.0,0,4760,8163.0,NaN,NaN,NaN,NaN
212,"Net Investments ($237,025 pledged as collatera...","Net Investments ($237,025 pledged as collatera...",780.0,209.0,5.36,34440.0,383210,542005,585628.0,NaN,NaN,NaN,NaN


In [217]:
last_4_column_empty = pd_csv.index[(pd_csv.iloc[: , -1].isnull()) & (pd_csv.iloc[: , -2].isnull()) & (pd_csv.iloc[: , -3].isnull())  & (pd_csv.iloc[: , -4].isnull()) ].to_list()

In [218]:
last_4_column_empty

[2, 32, 144, 172, 190, 194, 208, 212]

In [219]:
try:
    #Shifting the columns when fair value is empty
    pd_csv.loc[last_4_column_empty,obligor_cols[0]:] = pd_csv.loc[last_4_column_empty,obligor_cols[0]:].shift(4,axis=1)
except:
    print("Last 4 columns not present, Obligor Column shifting")

In [220]:
last_3_column_empty = pd_csv.index[(pd_csv.iloc[: , -1].isnull()) & (pd_csv.iloc[: , -2].isnull()) & (pd_csv.iloc[: , -3].isnull()) ].to_list()

In [221]:
pd_csv[(pd_csv.iloc[: , -1].isnull()) & (pd_csv.iloc[: , -2].isnull()) & (pd_csv.iloc[: , -3].isnull()) ]

,Levels,(Dollars in thousands),Obligor Name/Interest Rate Range,Security Type (all restricted unless otherwise noted),Acquisition Date,Maturity Date,No. of Invest.,% of Net Assets,Interest Rate,Original Cost of 2016 Acquisitions,Principal Outstanding,Cost,Fair Value


In [222]:
try:
    #Shifting the columns when fair value is empty
    pd_csv.loc[last_3_column_empty,obligor_cols[0]:] = pd_csv.loc[last_3_column_empty,obligor_cols[0]:].shift(3,axis=1)
except:
    print("Last 3 columns not present, Obligor Column shifting")

In [225]:
try:
    dollars_cols = [col for col in pd_csv.columns if 'Dollars in thousands' in col]
dollars_cols[0]

IndexError: list index out of range

In [230]:
pd_csv[pd_csv[pd_csv.columns[1]].astype(str).str.contains("Various Other")]

,Levels,(Dollars in thousands),Obligor Name/Interest Rate Range,Security Type (all restricted unless otherwise noted),Acquisition Date,Maturity Date,No. of Invest.,% of Net Assets,Interest Rate,Original Cost of 2016 Acquisitions,Principal Outstanding,Cost,Fair Value
38,Medallion Loans,Various Other && ##,4.75% to 11.50%,Term Loan,04/28/08 to 07/30/15,07/01/15 to 09/01/23,11.0,0,7.28,NaN,997.0,985.0,780.0
189,Investment Castings,Various Other #,**,* Various,09/10/98 to 7/24/15,None to 2/5/23,12,*,2.29,NaN,250.0,1366.0,1309.0
207,Investment Castings,Various Other #,**,* Various,09/10/98 to 7/24/15,None to 2/5/23,12,*,2.29,NaN,250.0,1366.0,1309.0


In [231]:
#Dropping everything after the last various other rows
try:
    various_other_list = pd_csv.index[pd_csv[pd_csv.columns[1]].astype(str).str.contains("Various Other")].to_list()[-1]
    pd_csv = pd_csv.loc[:various_other_list]
except:
    print("Various others not found")

In [232]:
pd_csv = pd_csv.fillna("")
try:
    for col in pd_csv.columns:
        # print("Column", col)
        pd_csv[col] = pd_csv[col].astype(str).str.replace(r"\(\d+\)", "", regex=True)
except:
    print("Bracket digits not removed")

In [233]:
pd_csv

,Levels,(Dollars in thousands),Obligor Name/Interest Rate Range,Security Type (all restricted unless otherwise noted),Acquisition Date,Maturity Date,No. of Invest.,% of Net Assets,Interest Rate,Original Cost of 2016 Acquisitions,Principal Outstanding,Cost,Fair Value
2,Medallion Loans,New York,,,,,386.0,71,3.76,12977.0,206576.0,206404.0,200440.0
3,Medallion Loans,New York,Real Cab Corp ##,Term Loan,07/20/07,07/20/17,1.0,1,2.81,,2545.0,2545.0,2545.0
4,Medallion Loans,New York,Real Cab Corp,Term Loan,08/19/14,07/20/17,1.0,*,3.31,,481.0,481.0,481.0
5,Medallion Loans,New York,Real Cab Corp ##,Term Loan,07/20/07,07/20/17,1.0,*,2.81,,350.0,350.0,350.0
6,Medallion Loans,New York,Sean Cab Corp ##,Term Loan,12/09/11,11/23/18,1.0,1,4.63,,3297.0,3297.0,3297.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,Investment Castings,Envirnomental Consulting Services,"Northern Technologies, LLC",7.7% of LLC units,01/29/16,None,1,*,0.0,,300.0,300.0,300.0
204,Investment Castings,Paper Tapes Manufacturer,"Liberty Paper Products Acquisition, LLC",100% of Series A preferred LLC units - 12% total,06/09/16,None,1,*,0.0,,350.0,350.0,350.0
205,Investment Castings,Surgical Supplies Manufacturer,"Respiratory Technologies, Inc.","Warrant for 38,000 Common shares",04/25/12,5th anniversary of note paid in full,1,*,0.0,0.0,0.0,,
206,Investment Castings,Hobbyists’ Supplies Merch. Wholesale,"Classic Brand, LLC","Warrant for 300,000 Class A units",01/08/16,01/08/26,1,*,0.0,,0.0,0.0,0.0


In [ ]:
pd_csv.to_csv("Testing1.csv")